# Klasifikasi Limbah

------

## Data Cleaning

In [1]:
import os
import numpy as np
import pandas as pd
import lxml.html as lh

In [2]:
# Bagian buat ngebuka / ngebaca dataset "data-limbah.html"
# dataset diluar folder repository soalnya mengandung privasi

# Ganti variable path sesuai dengan lokasi dataset
# dataset ditaro di variable table
path = "/Users/bunny/myGit/DataStore/Limbah/"
table = ""

for ls in os.listdir(path):
    if ls.endswith(".html"):
        print (ls)
        with open(path + ls, 'r') as file:
            table = file.read()

data-limbah.html


In [3]:
# convert string ke html
doc = lh.fromstring(table)

# Ekstrak baris pada table
# <tr>
tr = doc.xpath("//tr")

In [4]:
# Ekstrak nama kolom di row pertama
# nama kolom dimasukin ke variable feature_name
feature_name = []

for td in tr[0]:
    feature_name.append(td.text_content())

# Posisi kolom yang digunakan [2, 13, 14, 15, 17]
# Nama Pengirim, Kode Limbah, Jenis Limbah, Karakter Limbah, Kemasan
feature_name = [feature_name[2]] + feature_name[13:16] + [feature_name[17]]

In [5]:
# Ekstrak data
data = []

for i in range(1, len(tr[1:])):
    tr_ = tr[i]
    ccol = 0
    data_tmp = []
    
    for td in tr_:
        # Filter kolom yang digunakan
        if ccol == 2 or (ccol >= 13 and ccol <= 17) and ccol != 16:
            data_tmp.append(td.text_content())
        
        # Menghitung kolom
        ccol += 1
    
    data.append(data_tmp)

In [6]:
# Cek jumlah data
len(data)

10269

In [7]:
# Mumbuat dataframe
df = pd.DataFrame(data, columns=feature_name)

In [8]:
# Menghapus duplikat berdasarkan Nama Pengirim dan Kode Limbah
# Nama Pengirim memiliki banyak data dengan Kode Limbah yang sama
df = df.drop_duplicates()
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 5 columns):
Nama Pengirim      1020 non-null object
Kode Limbah        1020 non-null object
Jenis Limbah       1020 non-null object
Karakter Limbah    1020 non-null object
Kemasan            1020 non-null object
dtypes: object(5)
memory usage: 39.9+ KB


In [9]:
# Menghapus kolom Nama Pengirim (privasi)
df = df.drop(["Nama Pengirim"], axis=1)

In [10]:
# Sampel hasil ekstraksi
df.sample(3)

,Kode Limbah,Jenis Limbah,Karakter Limbah,Kemasan
447,A307-1,Sludge dari proses produksi dan fasilitas peny...,Beracun (toxic - T),IBC Tank
764,B109d,Filter bekas dari fasilitas pengendalian pence...,Beracun (toxic - T),Kemasan Karton atau plastik
355,B105d,Minyak pelumas bekas antara lain minyak peluma...,Beracun (toxic - T),Drum Logam


In [11]:
# Load dataset kode berdasarkan PP
kode = pd.read_csv("./Kode/kode-limbah.csv")
kode.sample(3)

,Kode,Sumber,Bahaya
722,B341-2,Spesifik Umum,2
363,A2300,?,1
192,A2129,?,1


In [12]:
# Load dataset bentuk
bentuk = pd.read_csv("./Kode/bentuk.csv")
bentuk.sample(3)

,Jenis,Bentuk
16,Residu proses produksi atau reaksi,Cair
31,Steel slag,Padat
4,Sludge proses produksi yang meliputi manufactu...,Padat


In [13]:
# Menggabungkan dataframe
df = df.join(kode.set_index("Kode"), on="Kode Limbah")
df = df.join(bentuk.set_index("Jenis"), on="Jenis Limbah")

In [14]:
# Mengubah nilai bentuk berdasarkan karakter
# Cairan Mudah menyala (ignitable - I) / Padatan Mudah menyala (ignitable - I)
# Beberapa tidak sesuai
df.loc[df["Karakter Limbah"].str.startswith("Cair"), "Bentuk"] = "Cair"
df.loc[df["Karakter Limbah"].str.startswith("Padat"), "Bentuk"] = "Padat"

In [15]:
# Menampilkan data dengan bentuk "Cair & Padat"
df = df[df["Bentuk"] != "Cair & Padat"]

In [16]:
# Mengubah nilai "Karakter Limbah"
# Cairan Mudah menyala (ignitable - I) / Padatan Mudah menyala (ignitable - I)
# Menjadi "I"
df["Karakter Limbah"] = df["Karakter Limbah"].str[-2]

In [17]:
# Filter dan mengurutkan kolom sesuai
df = df[["Kode Limbah", "Bentuk", "Karakter Limbah", "Sumber", "Bahaya", "Kemasan"]]
# df = df[["Kode Limbah", "Karakter Limbah", "Sumber", "Bahaya", "Kemasan"]]

In [18]:
# Mengganti tipe kolom (disesuaikan)
df.iloc[:, 1:] = df.astype("category")

In [19]:
# Export dataframe menjadi csv
# df.to_csv("./data/limbah.csv", index=False)

## Data Exploration

In [20]:
df.sample(3)

,Kode Limbah,Bentuk,Karakter Limbah,Sumber,Bahaya,Kemasan
279,B108d,Cair,T,Tidak Spesifik,2,Jumbo Bag
27,B323-1,Padat,T,Spesifik Umum,2,Tangki
942,A337-1,Cair,X,Spesifik Umum,1,Kemasan Karton atau plastik


In [21]:
df["Bentuk"].value_counts()

Padat    690
Cair     318
Name: Bentuk, dtype: int64

In [22]:
df["Karakter Limbah"].value_counts()

T    793
I    135
X     42
C     33
E      5
Name: Karakter Limbah, dtype: int64

In [23]:
df["Sumber"].value_counts()

Tidak Spesifik     525
Spesifik Umum      371
Spesifik Khusus    111
?                    1
Name: Sumber, dtype: int64

In [24]:
df["Bahaya"].value_counts()

2    630
1    378
Name: Bahaya, dtype: int64

In [25]:
df["Kemasan"].value_counts()

Drum Logam                      281
Tanpa Kemasan                   218
Jumbo Bag                       153
Kemasan Karton atau plastik     121
Box Besi                         51
Tangki                           40
Karung Kain, plastik, kertas     38
Drum Plastik                     33
IBC Tank                         25
R.O Container                    20
Wheel Bin                        10
Jerycan                          10
Drum kayu                         5
Wadah Hopper                      1
Gerbong Barang                    1
                                  1
Name: Kemasan, dtype: int64

In [26]:
# Remove label yang kemunculannya kurang dari 100
# df[~df["Kemasan"].isin(df["Kemasan"].value_counts()[df["Kemasan"].value_counts() < 100].index)]

### Outlier

In [27]:
df = df[df["Sumber"] != '?']

In [28]:
df = df[df["Kemasan"] != ""]

**Note**. 

1. Kolom bentuk punya 12 data bernilai "Cair & Padat" (hapus atau ubah)
2. Outlier nilai "?" dan kemasan tanpa nama "" muncul sekali (hapus atau ubah)
3. Nilai tidak seimbang

## Classification

In [29]:
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

In [30]:
label_bentuk = LabelEncoder()
label_karakter = LabelEncoder()
label_sumber = LabelEncoder()
label_bahaya = LabelEncoder()
label_kemasan = LabelEncoder()

In [31]:
label_bentuk.fit(df["Bentuk"])
label_karakter.fit(df["Karakter Limbah"])
label_sumber.fit(df["Sumber"])
label_bahaya.fit(df["Bahaya"])
label_kemasan.fit(df["Kemasan"])

LabelEncoder()

In [32]:
df["Kode Bentuk"] = label_bentuk.transform(df["Bentuk"])
df["Kode Karakter"] = label_karakter.transform(df["Karakter Limbah"])
df["Kode Sumber"] = label_sumber.transform(df["Sumber"])
df["Kode Bahaya"] = label_bahaya.transform(df["Bahaya"])
df["Kode Kemasan"] = label_kemasan.transform(df["Kemasan"])

In [33]:
X = df[["Kode Bentuk", "Kode Karakter", "Kode Sumber", "Kode Bahaya"]].values
y = df["Kode Kemasan"]

In [34]:
clf = MultinomialNB()
clf.fit(X, y)

# Support Vector Machine
# clf = SVC(gamma="auto")
# clf.fit(X, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [35]:
print (cross_val_score(clf, X, y, cv = 5))

[0.39805825 0.27722772 0.26237624 0.38888889 0.41414141]


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:626: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
